In [1]:
import pathlib
import subprocess
import os
import sys
import time

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [3]:

PRJ='PRJNA686836'
PRJ_OUT_PATH=f'/mnt/1TB_0/Data/Assembly/{PRJ}/'

de_novo='MEGAHIT'
contig_mer='final'
CONTIGS_NAME='MEGAHIT'
contigs_base_folder='megahit_default'

NUM_THREADS=8

DIAMOND_SENSITIVITY='--sensitive'
LOG_OPTION='--quiet'

sys.path.insert(0, '/mnt/1TB_ssd/Data/diamond/')
os.chdir(sys.path[0])

In [4]:
#PRJ='PRJNA573298'
#SRAs=['SRR10168373','SRR10168374','SRR10168375', 'SRR10168376','SRR10168377', 'SRR10168378'\
#        'SRR10168379','SRR10168380','SRR10168381','SRR10168382','SRR10168383',\
#      'SRR10168384','SRR10168385', 'SRR10168386','SRR10168387','SRR10168388',\
#      'SRR10168389','SRR10168390','SRR10168391','SRR10168392','SRR10168393']

SRAs=['SRR13285085']

In [5]:
### Index

In [6]:
def create_fq(contigsfa, contigs):
    try:
        cmd= f"~/apps/bbmap/reformat.sh in={contigsfa} out={contigs}"
        subprocess.check_call(cmd, shell=True)
    except subprocess.CalledProcessError as e:
        raise RuntimeError("command '{}' return with error (code {}): {}".format(e.cmd, e.returncode, e.output))

In [7]:
def run_diamond(contigs_fq, diamond_path, sra, sensitivity):
    assert os.path.isfile(contigs_fq)
    sens_txt=sensitivity
    if '--' in sensitivity: 
        sens_txt=sensitivity.split('--')[1]
    if de_novo=='MEGAHIT':
        daa_file= f"{diamond_path}{sra}_{CONTIGS_NAME}_{contig_mer}_{sens_txt}.daa"
    elif de_novo=='coronaspades':
        daa_file= f"{diamond_path}{sra}_coronaspades_{coronaspades_code}_{sens_txt}.daa"
    try:
        cmd = f"diamond blastx -d nr -q {contigs_fq} -o {daa_file} {sensitivity} -f 100 --threads {NUM_THREADS} {LOG_OPTION}"
        subprocess.check_call(cmd, shell=True)
    except subprocess.CalledProcessError as e:
        raise RuntimeError("command '{}' return with error (code {}): {}".format(e.cmd, e.returncode, e.output))

In [8]:
def workflow():
    start_time=time.time()
    idx=0
    for sra in SRAs:
        print(sra)
        start_loop=time.time()
        if de_novo=='MEGAHIT':
            #data_path=PRJ_OUT_PATH+sra+f'/{contigs_base_folder}/intermediate_contigs/'
            data_path=PRJ_OUT_PATH+sra+f'/metaxa2/{contigs_base_folder}/'
            contigs=data_path+f'{contig_mer}.contigs.fq'
            contigsfa=data_path+f'{contig_mer}.contigs.fa'
        elif de_novo=='coronaspades':
            data_path=PRJ_OUT_PATH+sra+'/metaxa2/coronaspades_default/'
            contigs=data_path+f'{coronaspades_name}.fq'
            contigsfa=data_path+f'{coronaspades_name}.fasta'
        if not os.path.isfile(contigsfa):
            print(f'Contigs file: {contigsfa} doesnt exist')
            raise Exception
        if not os.path.isfile(contigs):
            create_fq(contigsfa, contigs) 
        diamond_path=PRJ_OUT_PATH+sra+'/metaxa2/diamond/'
        if not os.path.isfile(diamond_path):
            pathlib.Path(diamond_path).mkdir(exist_ok=True)
        run_diamond(contigs, diamond_path, sra, DIAMOND_SENSITIVITY)
        print(f'Loop time {idx} {sra}: {time.time() - start_loop}')
        idx+=1
    print(f'Total time: {time.time() - start_time}')

In [9]:
workflow()

SRR13285085


RuntimeError: command 'diamond blastx -d nr -q /mnt/1TB_0/Data/Assembly/PRJNA686836/SRR13285085/metaxa2/megahit_default/final.contigs.fq -o /mnt/1TB_0/Data/Assembly/PRJNA686836/SRR13285085/metaxa2/diamond/SRR13285085_MEGAHIT_final_sensitive.daa --sensitive -f 100 --threads 8 --quiet' return with error (code 127): None

In [ ]:
#/mnt/8TB_0/Data/assembly/PRJNA607174/SRR11119759/megahit_default/final.contigs.fa
#/mnt/8TB_0/Data/Assembly/PRJNA607174/SRR11119759/megahit_default/final.contigs.fa